# Lattice Dynamics and Density Functional Perturbation Theory - Harmonic Level

The script 'thermodyanmics.py' is a comprehensive module for lattice dynamics based on harmonic and quasi-harmonic level of theory. For simplicity, this tutorial is focused only on harmonic phonons. For quasi-harmonic part, please refer to 'thermodynamics-QHA'.

## Class Mode
The basic definitions of a single vibrational mode at the given q point. It can be used for both HA and QHA levels of theory. Mode objects cannot be used for writing data output and calculating the thermodyanmics of the whole system. It can only be used to calculate the thermodynamics of a single phonon. Therefore, a 'shell' (e.g., 'Harmonic' object, see below) is needed to sum up individual phonons. For simplicity, only the attributes for HA are kept.

### HA-specific methods
**\_\_init\_\_(rank=0, frequency=[], volume=[], eigenvector=[])**  
Initialization. Required input: rank, frequency(as 1D numpy array or list), volume(as 1D numpy array or list), eigenvector(as 3D numpy array or list).

**get_zp_energy()**  
Get the zero-point energy of a single mode. Required input: None. Unit: KJ/mol cell

The zero-point energy is calculated based on the following equation: 

$$E_{zp,i,\mathbf{q}}=\frac{1}{2}\hbar\omega_{i,\mathbf{q}}$$

**get_U_vib(temperature=298.15)**  
Get the vibrational contribution to internal energy of a single mode. Required input: temperature. Zero point energy is included. Unit: KJ/mol cell. When Temperature = 0K, U_vib = zero point energy.

U_vib is calculated based on the following equation: 

$$U_{vib,i,\mathbf{q}}(T) = E_{zp,i,\mathbf{q}} + \frac{\hbar\omega_{i,\mathbf{q}}}{\exp{\big(\frac{\hbar\omega_{i,\mathbf{q}}}{k_{B}T}}\big) - 1}$$

**get_entropy(temperature=298.15)**  
Get the vibrational entropy of a single mode. Required input: temperature. Unit: KJ/mol cell*K. When Temperature = 0K, entropy = 0.

Entropy is calculated based on the following equation:

$$S_{i,\mathbf{q}}(T) = k_{B}\bigg\{\frac{\hbar\omega_{i,\mathbf{q}}}{k_{B}T\big[\exp{\big(\frac{\hbar\omega_{i,\mathbf{q}}}{k_{B}T}\big) - 1\big]}} - \ln{\bigg[1 - \exp{\bigg(-\frac{\hbar\omega_{i,\mathbf{q}}}{k_{B}T}\bigg)}\bigg]}\bigg\}$$

**get_C_v(temperature=298.15)**  
Get the contribution to constant volume specific heat of a single mode. Required input: temperature. Unit: KJ/mol cell*K. When Temperature = 0K, C_v = 0.

C_v is calculated based on the following equation:

$$C_{V,i,\mathbf{q}}(T) = \frac{(\hbar\omega_{i,\mathbf{q}})^{2}}{k_{B}T^{2}}\frac{\exp{\big(\frac{\hbar\omega_{i,\mathbf{q}}}{k_{B}T}\big)}}{\bigg[\exp{\big(\frac{\hbar\omega_{i,\mathbf{q}}}{k_{B}T}\big)} - 1\bigg]^{2}}$$

### HA-specific attributes
- **self.rank**, int, \_\_init\_\_, The rank of mode object starting from 1.  
- **self.ncalc**, int, \_\_init\_\_, The number of sampling (HA) calculations.  
- **self.volume**, ncalc * 1 numpy array, \_\_init\_\_, Volumes of sampling calculations. Unit: Angstrom^3.  
- **self.frequency**, ncalc * 1 numpy array, \_\_init\_\_, Angular frequencies of the mode. Unit: THz. Note: the angular frequency $\omega_{i}=2\pi\nu_{i}$.  
- **self.eigenvector**, ncalc * natom * 3 numpy array, \_\_init\_\_, Eigenvectors of the mode. Unit: Angstrom.  
- **self.zp_energy**, float, get_zp_energy, Zero point energy of the mode. Unit: KJ/mol cell.
- **self.U_vib**, float, get_U_vib, Vibration contribution to internal energy, including zero-point energy. Unit: KJ/mol cell.  
- **self.entropy**, float, get_entropy, Entropy of the mode. Unit: KJ/mol cell*K.  
- **self.C_v**, float, get_C_v, Constant volume specific heat. Unit: KJ/mol cell*K.  

For harmonic cases, ncalc = 1, and self.eigenvector is not needed.

In [1]:
class Mode:
    """
    Class Mode - store important information for a given vibrational mode. Can be used for
    a single harmonic phonon calculation and multiple calculations on the same system.
    
    Initialization:
        self.rank, __init__, The rank of the mode object. Start from 1.
        self.ncalc, __init__, The number of harmonic frequency calculations.
        self.frequency, __init__, Angular frequencies of the mode. Unit: THz
        self.volume, __init__, Cell volumes of harmonic calculations. Unit: Angstrom^3
        self.eigenvector, __init__, Corresponding eigenvectors. Unit: Angstrom
    
    Limited to ncalc = 1 cases:
        self.zp_energy, get_zp_energy, Zero point energy of the mode. Unit: KJ/mol cell
        self.U_vib, get_U_vib, Vibration contribution to internal energy, including
                    zero-point energy. Unit: KJ/mol cell
        self.entropy, get_entropy, Entropy of the mode. Unit: KJ/mol cell*K
        self.C_v, get_C_v, Constant volume specific heat. Unit: KJ/mol cell*K
    """

    def __init__(self, rank=0, frequency=[], volume=[], eigenvector=[]):
        """
        Input:
            rank, int, The rank of the mode object, from 1.
            frequency, ncalc * 1 array / list, Angular frequencies of the mode. Unit: THz
            volume, ncalc * 1 array / list, Lattice volumes of harmonic calculations.
                    Unit: Angstrom^3
            eigenvector, ncalc * natom * 3 array / list, Corresponding eigenvectors.
                         Unit: Angstrom
        Output:
            self.rank
            self.ncalc, int, The number of harmonic calculations
            self.frequency
            self.volume
            self.eigenvector
        """
        import numpy as np

        self.rank = rank
        self.ncalc = len(frequency)
        self.frequency = np.array(frequency, dtype=float) * 2 * np.pi
        self.volume = np.array(volume, dtype=float)
        self.eigenvector = np.array(eigenvector, dtype=float)

    def get_zp_energy(self):
        """
        Get the zero-point energy of a single mode. Limited to ncalc = 1 cases.
        
        Input:
            -
        Output:
            self.zp_energy, float, Zero-point energy of a given mode. Unit: KJ/mol cell
        """
        import numpy as np
        
        if self.ncalc > 1:
            print('Error: This modulus is limited to a single frequency calculation.')
            return
        
        hbar_freq = self.frequency[0] * 6.022141 * 6.626070E-2 / 2 / np.pi
        self.zp_energy = 0.5 * hbar_freq
        
        return self.zp_energy
        
    def get_U_vib(self, temperature=298.15):
        """
        Get the vibration contribution to internal energy of a single mode. Limited to
        ncalc = 1 cases. U_vib includes zero-point energy.
        
        Input:
            temperature: float, the temperature where the thermal contribution is computed.
        Output:
            self.U_vib, float, Vibration contribution to internal energy of a given mode.
            Unit: KJ/mol cell
        """
        import numpy as np
        
        if self.ncalc > 1:
            print('Error: This modulus is limited to a single frequency calculation.')
            return
        
        if not hasattr(self, 'zp_energy'):
            self.get_zp_energy()
            
        if temperature == 0:
            self.U_vib = self.zp_energy
            return self.U_vib
        
        hbar_freq = self.frequency[0] * 6.022141 * 6.626070E-2 / 2 / np.pi
        kb_t = 1.380649E-3 * 6.022141 * temperature
        expon = np.exp(hbar_freq / kb_t)
        self.U_vib = self.zp_energy + hbar_freq / (expon - 1)
        
        return self.U_vib
    
    def get_entropy(self, temperature):
        """
        Get the entropy of a single mode. Limited to ncalc = 1 cases.
        
        Input:
            temperature: float, the temperature where the thermal contribution is computed.
        Output:
            self.entropy, float, The entropy of a given mode. Unit: KJ/mol cell*K
        """
        import numpy as np
        
        if self.ncalc > 1:
            print('Error: This modulus is limited to a single frequency calculation.')
            return
        
        if temperature == 0:
            self.entropy = 0
            return self.entropy

        hbar_freq = self.frequency[0] * 6.022141 * 6.626070E-2 / 2 / np.pi
        kb_t = 1.380649E-3 * 6.022141 * temperature
        expon = np.exp(hbar_freq / kb_t)
        entS = kb_t * (hbar_freq / kb_t / (expon - 1) - np.log(1 - 1 / expon))
        self.entropy = entS / temperature

        return self.entropy
        
    def get_C_v(self, temperature):
        """
        Get the constant volume specific heat of a single mode. Limited to ncalc = 1 cases.
        
        Input:
            temperature: float, the temperature where the thermal contribution is computed.
        Output:
            self.C_v, float, The constant volume specific heat of a given mode.
            Unit: KJ/mol cell*K
        """
        import numpy as np
        
        if self.ncalc > 1:
            print('Error: This modulus is limited to a single frequency calculation.')
            return
        
        if temperature == 0:
            self.C_v = 0
            return self.C_v
        
        hbar_freq = self.frequency[0] * 6.022141 * 6.626070E-2 / 2 / np.pi
        kb_t = 1.380649E-3 * 6.022141 * temperature
        expon = np.exp(hbar_freq / kb_t)
        
        self.C_v = hbar_freq**2 / kb_t / temperature * expon / (expon - 1)**2
        
        return self.C_v


## Class Harmonic
Inherited from Freq_output class, because details of an HA calculation is included in a single .out file. Thermodynamic-specific attributes are added, such as summing up phonons.

### Methods
**\_\_init\_\_(output_name, temperature=[298.15], write_out=True, filename='HA-thermodynamics.dat')**  
Initialisation. Inputs: temperature(as 1D array or list), write_out, filename. If write_out == True, HA thermodynamic properties will be automatically calculated and printed to the output file.

**phonon_sumup(temperature=None)**  
Not an independent method. Sum up the contribution of each phonon to get the HA thermodynamic properties of the system at a given temperature. Contributions from the first 3 translational modes are neglected. When temperature is 'None', only zero-point energy will be calculated. When a temperature is given, U_vib, entropy and C_v are calculated.

**thermodynamics()**  
Calculate the thermodyanmics of the system within the given temperature range (self.temperature), including: Helmholtz free energy, zero-point energy, U_vib, entropy and C_v.

The vibrational Helmholtz free energy is given by:

$$F_{vib}(T) = E_{zp} + U_{vib}(T) - TS(T)$$

**print_results(file)**  
Print the HA thermodynamics parameters into a text file object 'file'. Launched when 'write_out' is set to be 'True' during initialisation. File format:

```
ELECTRONIC ENERGY = edft           KJ/MOL
CELL VOLUME =    volume        A^3, =  volume      CM^3/MOL
LATTICE PARAMETERS
           A           B           C       ALPHA        BETA       GAMMA
           a           b            c      alpha        beta       gamma

HARMONIC THERMODYNAMICS AT QPOINT #    rank of qpoint

  ZERO POINT ENERGY =   zp_energy  KJ/MOL

  T(K)     U_VIB(KJ/MOL) ENTROPY(KJ/MOL*K)     C_V(KJ/MOL*K) HELMHOLTZ(KJ/MOL)
 Tempt             U_vib           entropy               C_v         Helmholtz

HARMONIC THERMODYNAMICS AT QPOINT #    rank of qpoint

  ZERO POINT ENERGY =   zp_energy  KJ/MOL

  T(K)     U_VIB(KJ/MOL) ENTROPY(KJ/MOL*K)     C_V(KJ/MOL*K) HELMHOLTZ(KJ/MOL)
 Tempt             U_vib           entropy               C_v         Helmholtz
```

### Attributes
Class Harmonic inherites all the attributes of the class Freq_output. New attributes are added for calculating and storing thermodynamic properties, including:

* **self.mode**, __init__, nqpoint * nmode array, List of mode objects at all qpoints.  
* **self.temperature**, __init__, nTempt * 1 array, The temperature range for HA thermodynamics.  
* **self.helmholtz**, thermodynamics, nqpoint * nTempt array, Helmholtz free energy of at all q points and the given temperature range, HA level. Unit: KJ/mol cell.  
* **self.U_vib**, thermodynamics, nqpoint * nTempt array, Vibrational contribution to internal energy at all q points and the given temperature range, HA level.  Unit: KJ/mol cell.  
* **self.zp_energy**, thermodynamics, nqpoint * 1 array, Zero-point energy at all q points, HA level.  Unit: KJ/mol cell.  
* **self.entropy**, thermodynamics, nqpoint * nTempt array, Entropy at all q points and the given temperature range, HA level.  Unit: KJ/mol cell*K.  
* **self.C_v**, thermodynamics, nqpoint * nTempt array, Constant volume specific heat at all q points and the given temperature range, HA level.  Unit: KJ/mol cell*K.  


In [2]:
from crystal_functions.lattice_dynamics.file_readwrite import Freq_output


class Harmonic(Freq_output):
    """
    Class Harmonic, inherited from the Freq_output class, with thermodynamic attributes.
    Used for harmonic phonon calclulations (single file).
    
    Inherited attributes:
        self.lattice
        self.edft
        self.nmode
        self.frequency
    
    New attributes:
        self.mode, __init__, The list of mode objects.
        self.temperature, __init__, The temperature range for HA thermodynamics.
        self.helmholtz, thermodynamics, Helmholtz free energy of the cell, at HA level.
                        Unit: KJ/mol cell
        * Following attributes are the summed up values of corresponding attributes of
          class Mode.
        self.U_vib, thermodynamics
        self.zp_energy, thermodynamics 
        self.entropy, thermodynamics
        self.C_v, thermodynamics
    """
    def __init__(self, output_name, temperature=[298.15],
                 write_out=True, filename='HA-thermodynamics.dat'):
        """
        Input:
            output_name, str, Name of the output file.
            Temperature, nTempt * 1 array / list, Temperatures where the thermodynamic
                         properties are computed. Unit: K
            write_out, bool, Wheter to print out HA thermodynamic properties.
            filename, str, Name of the printed-out file, used only if write_out = True.
        Output:
            self.temperature, nTempt * 1 array / list, Temperature range. Unit K
            self.mode, nqpoint * nmode list, List of mode objects at all qpoints.
            filename, text file, HA thermodynamic data. 'Thermodynamics' method will be
                      automatically executed and a file will be printed out, if
                      write_out = True.
        """
        import numpy as np

        super(Harmonic, self).__init__(output_name)
        self.temperature = np.array(temperature, dtype=float)
        self.get_edft()
        self.get_mode()
        self.clean_imaginary()
        
        vol = self.lattice.volume
        
        # Transfer the modes in self.freqency into lists of mode objects
        modes = []
        for qpoint in self.frequency:
            qmodes = []
            for m, freq in enumerate(qpoint):
                qmodes.append(Mode(rank=m + 1, frequency=[freq], volume=[vol]))

            modes.append(qmodes)
        
        self.mode = modes
        
        if write_out:
            self.thermodynamics()
            wtout = open(filename, 'w')
            self.print_results(file=wtout)
            wtout.close()

    def phonon_sumup(self, temperature=None):
        """
        Summing up inidival phonon modes at each q point. Translational modes with
        frequencies = 0 are skipped.
        
        Not a standalone method. For thermodynamics, use 'self.thermodyanmics' instead.
        
        Input:
            temperature, float, The temperature where the U_vib, entropy and C_v are
                         calculated. If = None, zero_point energy will be calculated.
        Output:
            zp_energy, float, Zero-point energy at a given q point. Returned if temperature
                       = None.
            U_vib, float, Vibrational contribution to internal energy at constant temperature
                   and given q point. Returned if temperature is given.
            entropy, float, Entropy at constant temperature and given q point. Returned if
                     temperature is given.
            C_v, float, Constant volume specific heat at constant temperature and given q
                 point. Returned if temperature is given.
        """
        import numpy as np
        
        if temperature == None:
            zp_energy = np.array([], dtype=float)
            for qpoint in self.mode:
                zp_energy_q = 0.
                # Remove the translational modes
                for mode in qpoint:
                    if not np.isnan(mode.frequency) and mode.frequency > 1e-5:
                        zp_energy_q += mode.get_zp_energy()
                    
                zp_energy = np.append(zp_energy, zp_energy_q)
                
            return zp_energy
        else:
            T = temperature
            U_vib = np.array([], dtype=float)
            entropy = np.array([], dtype=float)
            C_v = np.array([], dtype=float)
            for qpoint in self.mode:
                U_vib_q = 0.
                entropy_q = 0.
                C_v_q = 0.
                # Remove the translational modes
                for mode in qpoint:
                    if not np.isnan(mode.frequency) and mode.frequency > 1e-5:
                        U_vib_q += mode.get_U_vib(temperature=T)
                        entropy_q += mode.get_entropy(temperature=T)
                        C_v_q += mode.get_C_v(temperature=T)
        
                U_vib = np.append(U_vib, U_vib_q)
                entropy = np.append(entropy, entropy_q)
                C_v = np.append(C_v, C_v_q)
            
            return U_vib, entropy, C_v
        
    def thermodynamics(self):
        """
        Calculate the thermodynamic properties (zp_energy, U_vib, entropy, C_v and Helmholtz
        free energy) of the given system, at all qpoints and the whole temperature range.
        
        Input:
            -
        Output:
            self.helmholtz, nqpoint * nTempt numpy array, Helmholtz free energy.
                            Unit: KJ/mol cell
            self.zp_energy, nqpoint * 1 numpy array, Zero-point energy. Unit: KJ/mol cell
            self.U_vib, nqpoint * nTempt numpy array, Vibrational contribute to internal
                        energy. Unit: KJ/mol cell
            self.entropy, nqpoint * nTempt numpy array, Entropy. Unit: KJ/mol cell*K
            self.C_v, nqpoint * nTempt numpy array, Constant volume specific heat.
                      Unit: KJ/mol cell*K
        """
        import numpy as np
        
        self.zp_energy = self.phonon_sumup()
        self.U_vib = []
        self.entropy = []
        self.C_v = []
        self.helmholtz = []
        
        for T in self.temperature:
            U_vib_t, entropy_t, C_v_t = self.phonon_sumup(temperature=T)
            helm_t = -entropy_t * T + U_vib_t + self.edft
            
            self.U_vib.append(U_vib_t)
            self.entropy.append(entropy_t)
            self.C_v.append(C_v_t)
            self.helmholtz.append(helm_t)
        
        self.U_vib = np.array(self.U_vib, dtype=float).transpose()
        self.entropy = np.array(self.entropy, dtype=float).transpose()
        self.C_v = np.array(self.C_v, dtype=float).transpose()
        self.helmholtz = np.array(self.helmholtz, dtype=float).transpose()
            
        return self.helmholtz, self.zp_energy, self.U_vib, self.entropy, self.C_v

    def print_results(self, file):
        """
        Print a single output file for HA thermodynamics. Used if write_out=True.
        
        Input:
            file, file object obtained by 'open' command.
        Output:
            filename, text file.
        Format:
ELECTRONIC ENERGY = edft           KJ/MOL
CELL VOLUME =    volume        A^3, =  volume      CM^3/MOL
LATTICE PARAMETERS
           A           B           C       ALPHA        BETA       GAMMA
           a           b            c      alpha        beta       gamma

HARMONIC THERMODYNAMICS AT QPOINT #    rank of qpoint

  ZERO POINT ENERGY =   zp_energy  KJ/MOL

  T(K)     U_VIB(KJ/MOL) ENTROPY(KJ/MOL*K)     C_V(KJ/MOL*K) HELMHOLTZ(KJ/MOL)
 Tempt             U_vib           entropy               C_v         Helmholtz

HARMONIC THERMODYNAMICS AT QPOINT #    rank of qpoint

  ZERO POINT ENERGY =   zp_energy  KJ/MOL

  T(K)     U_VIB(KJ/MOL) ENTROPY(KJ/MOL*K)     C_V(KJ/MOL*K) HELMHOLTZ(KJ/MOL)
 Tempt             U_vib           entropy               C_v         Helmholtz
        """
        file.write('%19s%12.4e%10s\n' % ('ELECTRONIC ENERGY =', self.edft, 'KJ/MOL'))
        file.write('%-15s%12.6f%10s%12.6f%10s\n' % 
                   ('CELL VOLUME =', self.lattice.volume, ' A^3, =',
                    self.lattice.volume * 0.602214, 'CM^3/MOL'))
        file.write('%s\n' % 'LATTICE PARAMETERS')
        file.write('%12s%12s%12s%12s%12s%12s\n' % ('A', 'B', 'C',
                                                   'ALPHA', 'BETA', 'GAMMA'))
        file.write('%12.4f%12.4f%12.4f%12.4f%12.4f%12.4f\n\n' % 
                   (self.lattice.lattice.parameters[0:6]))
        
        for q in range(self.nqpoint):
            file.write('%35s%5i\n\n' % ('HARMONIC THERMODYNAMICS AT QPOINT #', q))
            file.write('%21s%12.6e%8s\n\n' %
                       ('ZERO POINT ENERGY =', self.zp_energy[q], 'KJ/MOL'))
            file.write('%6s%18s%18s%18s%18s\n' %
                       ('T(K)', 'U_VIB(KJ/MOL)','ENTROPY(KJ/MOL*K)', 'C_V(KJ/MOL*K)',
                        'HELMHOLTZ(KJ/MOL)'))
            for t, tempt in enumerate(self.temperature):
                file.write('%6.2f%18.6e%18.6e%18.6e%18.6e\n' %
                           (tempt, self.U_vib[q, t], self.entropy[q, t],
                            self.C_v[q, t], self.helmholtz[q, t]))
                
            file.write('\n')


## Tests
2 test are performed. One is based on $\Gamma$ point calculations and the other one is based on phonon dispersion calculations.

1. Thermodynamic properties at room temperature (298.15K), 0 pressure - For comparison with CRYSTAL17 output.  
2. Thermodynamic properties from 0K to 300K, with the interval of 20K - To illustrate the structure of output file and key information.

### Room temperature (298.15K), 0 pressure thermodyanmics
This part is used for comparison with CRYSTAL17 output file, in order to illustrate the equivalence of methods adopted here and the method of CRYSTAL17. $\Gamma$ point calculation of Form I paracetamol is used ('freqf1-r0.out' in the same directory). Reference data with the same environmental conditions can be found in the same file. No output file is generated. 

In [8]:
rm_condition = Harmonic('freqf1-r0.out', temperature=[298.15], write_out=False)
E_h, E_zp, U_vib, S, C_v = rm_condition.thermodynamics()

# Note: All attributes are lists.
print('DFT total energy (EL) = ', rm_condition.edft, 'KJ/mol')
print('Helmholtz free energy (EL+E0+ET-TS) = ', E_h[0, 0], ' KJ/mol')
print('Zero-point energy (E0) = ', E_zp[0], ' KJ/mol')
print('Vibrational contribution to interla energy - E0 (ET) = ', U_vib[0, 0] - E_zp[0], ' KJ/mol')
print('Entropy*Temperature (TS) = ', S[0, 0] * 298.15, ' KJ/mol')
print('Heat capacity = ', C_v[0, 0] * 1000, ' J/mol*K')
print('Entropy = ', S[0, 0] * 1000, ' J/mol*K')

DFT total energy (EL) =  -5402457.523570631 KJ/mol
Helmholtz free energy (EL+E0+ET-TS) =  -5400802.066049164  KJ/mol
Zero-point energy (E0) =  1759.1003801887073  KJ/mol
Vibrational contribution to interla energy - E0 (ET) =  107.20367473751753  KJ/mol
Entropy*Temperature (TS) =  210.8465334596594  KJ/mol
Heat capacity =  665.575318228324  J/mol*K
Entropy =  707.1827384191159  J/mol*K


References from 'freqf1-r0.out' (line 11703~11738) are attached below. Here are several findings:

1. Parameters derived from vibrational frequencies shows good agreement with reference data, with the error at the level of 10^-4 KJ/mol. The choice of physical constants and different unit conversion coefficients might lead to the discrepancy, where the physical constants play a major role (See below).  
2. The difference in DFT total energy lies in different coefficients for unit conversion. In CRYSTAL17 case, 1 Hartree = 2625.5 KJ/mol. In the current case, 1 Hartree = 2625.500256 KJ/mol.  
3. The PV term is neglected at room condition. Helmholtz free energy is the focus of this tutorial since the Harmonic class was initially developed for QHA fittings. To calculate Gibbs free energy at HA level, use this equation: $G(T,p) = F(T) + pV$

```
 *******************************************************************************

 HARMONIC VIBRATIONAL CONTRIBUTIONS TO THERMODYNAMIC FUNCTIONS AT GIVEN
 TEMPERATURE AND PRESSURE:

 (EL = ELECTRONIC ENERGY
  E0 = ZERO-POINT ENERGY
  ET = THERMAL CONTRIBUTION TO THE VIBRATIONAL ENERGY
  PV = PRESSURE * VOLUME
  TS = TEMPERATURE * ENTROPY)

                          AU/CELL             EV/CELL                 KJ/MOL
 EL            :   -2057.686915559598  -55992.507576455653    -5402456.23545757
 E0            :       0.670005954735      18.231788914567        1759.10038625


 *******************************************************************************

 THERMODYNAMIC FUNCTIONS WITH VIBRATIONAL CONTRIBUTIONS

 AT (T =  298.15 K, P =   0.10132500E+00 MPA):

                          AU/CELL             EV/CELL                 KJ/MOL
 ET            :       0.040831798552       1.111089725297         107.20387199
 PV            :       0.000017530544       0.000477030356           0.04602644
 TS            :       0.080307302905       2.185272809177         210.84679406
 ET+PV-TS      :      -0.039457973810      -1.073706053524        -103.59689564
 EL+E0+ET+PV-TS:   -2057.056367578673  -55975.349493594607    -5400800.73196695

 OTHER THERMODYNAMIC FUNCTIONS:

                      mHARTREE/(CELL*K)     mEV/(CELL*K)              J/(MOL*K)
 ENTROPY       :       0.269352013769       7.329440916241         707.18361249
 HEAT CAPACITY :       0.253504784422       6.898215882635         665.57671770

 *******************************************************************************
```

### Multiple q points, Multiple temperatures
Class Harmonic can calculate the HA thermodynamics at a set of q points in order to sample various cell sizes. As a test case illustrating the effectiveness of this, a phonon dispersion calculation is performed based on the $\Gamma$ point frequency data of Form I paracetamol crystal. `SCELPHONO` is set to be 1 to save computation time.

The reference data is 'f1-disp.out' - Frequencies at $\Gamma$ are different from the 'f1freq-r0.out' case due to unknown reasons. In this case, nqpoint = 140, nmode = 240, natom = 80. The temperature range is set to be 0K ~ 300K, with intervals of 20K. Since negative frequencies presents, warning messages will be printed. 

All the calculations are automatically executed. The output file is 'f1-disp-HA.dat'. 

In [11]:
import numpy as np

t = np.arange(0, 301, 30)
r0_freq = Harmonic('f1-disp.out', temperature=t, write_out=True, filename='f1-disp-HA.dat')